# Preparing Environment

In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas import DataFrame
from math import radians
import math
import numpy as np
from sklearn.model_selection import train_test_split
#from haversine import haversine
#from haversine import haversine_vector,Unit
import warnings; warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from datetime import datetime
from datetime import date
import calendar
distanceRange = 100
numberOfRows = 6
skillRateRange = 3

# Data Collection :

# Access and Import Files from the MongoDB Database

In [2]:
from pprint import pprint
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient("mongodb://78.47.77.101:27017/root")
db=client.admin
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
print(serverStatusResult)


{'host': 'kibana', 'version': '4.2.3', 'process': 'mongod', 'pid': 13184, 'uptime': 4947129.0, 'uptimeMillis': 4947129174, 'uptimeEstimate': 4947129, 'localTime': datetime.datetime(2020, 4, 18, 2, 42, 22, 998000), 'asserts': {'regular': 0, 'warning': 0, 'msg': 1, 'user': 706, 'rollovers': 0}, 'connections': {'current': 8, 'available': 51192, 'totalCreated': 8648, 'active': 1}, 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0}, 'priorityTakeover': {'called': 0, 'successful': 0}, 'catchUpTakeover': {'called': 0, 'successful': 0}, 'electionTimeout': {'called': 0, 'successful': 0}, 'freezeTimeout': {'called': 0, 'successful': 0}, 'numStepDownsCausedByHigherTerm': 0, 'numCatchUps': 0, 'numCatchUpsSucceeded': 0, 'numCatchUpsAlreadyCaughtUp': 0, 'numCatchUpsSkipped': 0, 'numCatchUpsTimedOut': 0, 'numCatchUpsFailedWithError': 0, 'numCatchUpsFailedWithNewTerm': 0, 'numCatchUpsFailedWithReplSetAbortPrimaryCatchUpCmd': 0, 'averageCatchUpOps': 0.0}, 'extra_info': {'note': 'fields var

In [3]:
#scores=5

In [4]:
#Distance_in_km=100

In [5]:
job_db=client.eande
print(job_db)

Database(MongoClient(host=['78.47.77.101:27017'], document_class=dict, tz_aware=False, connect=True), 'eande')


In [6]:
myCursor = job_db.skills.find( {} )
df_skills = DataFrame(list(job_db.skills.find( {} )))
df_JobProposal=DataFrame(list(job_db.JobProposal.find({})))
df_PostJob=DataFrame(list(job_db.PostJob.find({})))
df_JobSkills=DataFrame(list(job_db.JobSkills.find({})))
df_UserLang=DataFrame(list(job_db.UserLang.find({})))
df_Company=DataFrame(list(job_db.Company.find({})))
df_users=DataFrame(list(job_db.users.find({})))
df_UserSkills=DataFrame(list(job_db.UserSkills.find({})))
df_UserCertification = DataFrame(list(job_db.UserCertification.find( {} )))
df_langs = DataFrame(list(job_db.langs.find( {} )))
df_roles = DataFrame(list(job_db.roles.find( {} )))


In [7]:
#COUNTING THE USER JOB EXPERIENCE
df_UserJobExperience=DataFrame(list(job_db.UserJobExperience.find({})))
df_UserJobExperience['start_date']=pd.to_datetime(df_UserJobExperience['start_date'],format ='%d-%b-%Y %H:%M:%S')
df_UserJobExperience['end_date']=pd.to_datetime(df_UserJobExperience['end_date'],format ='%d-%b-%Y %H:%M:%S')
df_UserJobExperience['Experience in Months'] =(df_UserJobExperience.end_date.dt.year - df_UserJobExperience.start_date.dt.year) * 12 +(df_UserJobExperience.end_date.dt.month - df_UserJobExperience.start_date.dt.month)
df_UserJobExperience.head()

,_id,description,status,created_at,updated_at,__v,user_id,org_name,end_date,title,start_date,Experience in Months
0,5e5ab8a09da33c5135fa5a35,Investor Optimization Representative,1,2018-07-22 03:00:12,2014-05-16 18:09:18,0.0,7d321f5c-36af-42ba-adb1-51f0c354505c,Reichel LLC,2020-04-12 18:00:00,Customer Metrics Analyst,2019-04-12 08:00:00,12
1,5e5ab8a09da33c5135fa5a36,Dynamic Security Director,1,2011-11-13 06:50:47,2015-08-13 19:43:19,0.0,326791f7-69fc-4bf8-bfb2-4efaac0856c1,"Anderson, Mitchell and Maggio",2020-04-12 18:00:00,Global Markets Officer,2019-04-12 08:00:00,12
2,5e5ab8a09da33c5135fa5a37,Human Security Supervisor,1,2011-11-22 23:13:59,2019-02-13 05:41:22,0.0,65dac957-39bf-4d4a-b011-5e00f9d2417d,Lowe-Bradtke,2020-04-12 18:00:00,Dynamic Data Facilitator,2020-04-12 08:00:00,0
3,5e5ab8a09da33c5135fa5a38,Forward Response Facilitator,1,2010-02-05 07:10:49,2017-04-30 05:37:14,0.0,a74932df-7e7d-4572-a4f6-f738de6c2d31,"Gutkowski, Marvin and McCullough",2020-04-12 18:00:00,Global Response Specialist,2020-04-12 08:00:00,0
4,5e5ab8a09da33c5135fa5a39,National Tactics Administrator,1,2013-12-27 09:28:22,2017-10-23 16:03:16,0.0,c549ac2b-8c60-40f7-802a-0edde18cc9d3,Kirlin-Willms,2020-04-12 18:00:00,Dynamic Tactics Strategist,2020-04-12 08:00:00,0


# Merging dataframes

In [8]:
df1=pd.merge(df_PostJob,df_JobSkills,left_on='_id',right_on='job_id',how='left')
df1.head().transpose()

,0,1,2,3,4
_id_x,e3b3470d-76f6-45c1-8afb-51035151e21c,0e4db772-d915-4a4f-ad7e-65885a361744,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,96d0688a-204d-4529-9da5-34f653fec203,599334aa-c6d9-4306-aff9-5faccaf08c6f
start_date,12-APR-2019 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00
end_time,18:00:00,18:00:00,18:00:00,18:00:00,18:00:00
service_id,Somalia,Pakistan,Myanmar,Mauritania,Greenland
total_cost,70,80,80,80,90
status,1,1,1,1,1
job_title,Human Data Analyst,3:Casinos:,3:Casinos:,Customer Metrics Analyst,3:Casinos:
end_date,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00
total_hiring,3,3,3,3,3
hourly_cost,13,10,13,11,10


In [9]:
df2_jobs=pd.merge(df1,df_Company,on='company_id',how='left')
df2_jobs.head().transpose()

,0,1,2,3,4
_id_x,e3b3470d-76f6-45c1-8afb-51035151e21c,0e4db772-d915-4a4f-ad7e-65885a361744,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,96d0688a-204d-4529-9da5-34f653fec203,599334aa-c6d9-4306-aff9-5faccaf08c6f
start_date,12-APR-2019 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00,12-APR-2020 08:00:00
end_time,18:00:00,18:00:00,18:00:00,18:00:00,18:00:00
service_id,Somalia,Pakistan,Myanmar,Mauritania,Greenland
total_cost,70,80,80,80,90
status_x,1,1,1,1,1
job_title,Human Data Analyst,3:Casinos:,3:Casinos:,Customer Metrics Analyst,3:Casinos:
end_date,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00,12-APR-2020 18:00:00
total_hiring,3,3,3,3,3
hourly_cost,13,10,13,11,10


In [10]:
df2_jobs=df2_jobs[['_id_x','rate_x','status_x','total_cost','job_title','company_id','company_name','lat','postal','long','skill_id','job_description']]

df2_jobs.columns=['Job_Id','Skill_rate','Job_Status','Total_salary','Job_Title','Company_id','Company_name','Company_lat','Postal','Company_long','Skill_Id','Job_Description']
df2_jobs.head()

,Job_Id,Skill_rate,Job_Status,Total_salary,Job_Title,Company_id,Company_name,Company_lat,Postal,Company_long,Skill_Id,Job_Description
0,e3b3470d-76f6-45c1-8afb-51035151e21c,4,1,70,Human Data Analyst,ee37daa8-1473-4d6e-ab08-aea675ba781a,Gutkowski-Bosco,34.21667,80712,-72.616669,e976c9af-784a-4874-8e3a-2b13c3017ecf,Digitized directional matrix
1,0e4db772-d915-4a4f-ad7e-65885a361744,3,1,80,3:Casinos:,6a9ecc7a-4d95-4861-b545-e1fa40c31be1,Harris-Hahn,16.216667,07445-5615,-72.616669,84f9eb38-dd84-42c0-b3b9-719c019a7f00,Progressive scalable customer loyalty
2,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,5,1,80,3:Casinos:,945c5726-2772-49b4-9c70-57da20458762,"Mueller, Leannon and D'Amore",40.216667,15033-7044,-72.616669,a2eae9ba-314e-418e-997a-909b7e365997,Focused web-enabled parallelism
3,96d0688a-204d-4529-9da5-34f653fec203,4,1,80,Customer Metrics Analyst,275e41f9-8450-49ab-b1d9-1233edc97327,Wilkinson-Boyer,96.216667,47054-3564,-72.616669,194ed753-2ebb-4451-95b8-a491a3cf49b4,Profound cohesive methodology
4,599334aa-c6d9-4306-aff9-5faccaf08c6f,2,1,90,3:Casinos:,80c7ba57-8dc3-4927-aae1-4e02167510b8,Von LLC,46.216667,08564,-72.616669,87853669-bbd5-49d0-a542-358024a83e16,Reverse-engineered scalable internet solution


In [11]:
df2_jobs["Skill_Id"].fillna("Null", inplace = True)
len(df2_jobs)
df2_jobs.isna().sum()

Job_Id             0
Skill_rate         0
Job_Status         0
Total_salary       0
Job_Title          0
Company_id         0
Company_name       0
Company_lat        0
Postal             0
Company_long       0
Skill_Id           0
Job_Description    0
dtype: int64

In [12]:
df3=pd.merge(df_users,df_UserSkills,left_on='_id',right_on='user_id',how='left')
df3.head()

,_id_x,__v_x,ip_address,updated_at_x,lat,created_at_x,status,email,password,dob,...,long,rate_x,role_id,_id_y,__v_y,skill_id,user_id,rate_y,created_at_y,updated_at_y
0,7d321f5c-36af-42ba-adb1-51f0c354505c,0.0,10.32.12.213,2018-07-22 03:00:12,46.216667,2014-05-16 18:09:18,1.0,kellen@zemlak.biz,98a04fa6-8c09-4e69-8aae-cf1303a3c601,1985-03-01,...,-72.616669,1,bafe6d67-1924-44f1-aea1-407eca5300c8,5e5ab8a09da33c5135fa564b,0.0,e976c9af-784a-4874-8e3a-2b13c3017ecf,7d321f5c-36af-42ba-adb1-51f0c354505c,1.0,2018-07-22 03:00:12,2014-05-16 18:09:18
1,326791f7-69fc-4bf8-bfb2-4efaac0856c1,0.0,10.32.12.213,2011-11-13 06:50:47,46.216667,2015-08-13 19:43:19,1.0,anahi@jewess.biz,228d443c-8b92-4d02-b6b9-09340e1adfb9,1985-09-01,...,-72.616669,4,fe5378ec-1932-41ac-98a9-1105fd644d7f,5e5ab8a09da33c5135fa564c,0.0,84f9eb38-dd84-42c0-b3b9-719c019a7f00,326791f7-69fc-4bf8-bfb2-4efaac0856c1,4.0,2011-11-13 06:50:47,2015-08-13 19:43:19
2,65dac957-39bf-4d4a-b011-5e00f9d2417d,0.0,10.32.12.213,2011-11-22 23:13:59,46.216667,2019-02-13 05:41:22,1.0,simone@gusikowski.biz,7d1c84c5-8489-474d-9232-0dad56a7a438,1995-09-01,...,-72.616669,3,471bc505-f698-40d5-a8ac-eec09be8c9f9,5e5ab8a09da33c5135fa564d,0.0,a2eae9ba-314e-418e-997a-909b7e365997,65dac957-39bf-4d4a-b011-5e00f9d2417d,3.0,2011-11-22 23:13:59,2019-02-13 05:41:22
3,a74932df-7e7d-4572-a4f6-f738de6c2d31,0.0,10.32.12.213,2010-02-05 07:10:49,46.216667,2017-04-30 05:37:14,1.0,arvilla@douglaskassulke.name,fd590e52-954f-40ec-9ac6-c85dc01e2150,2001-09-01,...,-72.616669,2,03c7dbc3-7be8-4f28-9223-0fb4444c49b4,5e5ab8a09da33c5135fa564e,0.0,194ed753-2ebb-4451-95b8-a491a3cf49b4,a74932df-7e7d-4572-a4f6-f738de6c2d31,2.0,2010-02-05 07:10:49,2017-04-30 05:37:14
4,c549ac2b-8c60-40f7-802a-0edde18cc9d3,0.0,10.32.12.213,2013-12-27 09:28:22,46.216667,2017-10-23 16:03:16,1.0,johann_schneider@grady.biz,7f66f005-9412-49ac-826b-1558db5606da,1985-09-01,...,-72.616669,2,0121d380-c0a8-4da5-b090-ec87f43246db,5e5ab8a09da33c5135fa564f,0.0,87853669-bbd5-49d0-a542-358024a83e16,c549ac2b-8c60-40f7-802a-0edde18cc9d3,2.0,2013-12-27 09:28:22,2017-10-23 16:03:16


In [13]:
len(df3)
df3.isna().sum()

_id_x              0
__v_x              0
ip_address         0
updated_at_x       0
lat                0
created_at_x       0
status             0
email              0
password           0
dob                0
first_name         0
last_name          0
salt               0
country_code       0
mobile_no          0
address            0
long               0
rate_x             0
role_id            0
_id_y           9000
__v_y           9000
skill_id        9000
user_id         9000
rate_y          9000
created_at_y    9000
updated_at_y    9000
dtype: int64

In [14]:
df4=pd.merge(df_UserLang,df_UserJobExperience,on='user_id',how='left')
df4.head()

,_id_x,created_at_x,updated_at_x,__v_x,lang_id,user_id,status_x,_id_y,description,status_y,created_at_y,updated_at_y,__v_y,org_name,end_date,title,start_date,Experience in Months
0,5e5ab8a19da33c5135fa8d1b,2019-11-22 02:57:28,2012-08-14 13:32:37,0.0,b60c3f90-5391-43a5-ab96-f6eb65dd80df,7d321f5c-36af-42ba-adb1-51f0c354505c,1,5e5ab8a09da33c5135fa5a35,Investor Optimization Representative,1,2018-07-22 03:00:12,2014-05-16 18:09:18,0.0,Reichel LLC,2020-04-12 18:00:00,Customer Metrics Analyst,2019-04-12 08:00:00,12
1,5e5ab8a19da33c5135fa8d1c,2010-08-12 12:41:15,2012-07-10 07:03:20,0.0,b769ad2b-a4e7-44f1-9a16-f9379e755ba4,326791f7-69fc-4bf8-bfb2-4efaac0856c1,1,5e5ab8a09da33c5135fa5a36,Dynamic Security Director,1,2011-11-13 06:50:47,2015-08-13 19:43:19,0.0,"Anderson, Mitchell and Maggio",2020-04-12 18:00:00,Global Markets Officer,2019-04-12 08:00:00,12
2,5e5ab8a19da33c5135fa8d1d,2017-06-13 19:54:06,2014-09-25 21:35:08,0.0,86582b97-988e-4c7b-8930-8e4a36be275a,65dac957-39bf-4d4a-b011-5e00f9d2417d,1,5e5ab8a09da33c5135fa5a37,Human Security Supervisor,1,2011-11-22 23:13:59,2019-02-13 05:41:22,0.0,Lowe-Bradtke,2020-04-12 18:00:00,Dynamic Data Facilitator,2020-04-12 08:00:00,0
3,5e5ab8a19da33c5135fa8d1e,2016-03-02 13:57:23,2012-06-06 13:18:57,0.0,a1d428c2-ea35-4270-89eb-030fa4348aa6,a74932df-7e7d-4572-a4f6-f738de6c2d31,1,5e5ab8a09da33c5135fa5a38,Forward Response Facilitator,1,2010-02-05 07:10:49,2017-04-30 05:37:14,0.0,"Gutkowski, Marvin and McCullough",2020-04-12 18:00:00,Global Response Specialist,2020-04-12 08:00:00,0
4,5e5ab8a19da33c5135fa8d1f,2018-06-01 08:46:34,2014-02-10 09:46:42,0.0,e18a078d-9576-4cdb-a619-3790430b85d6,c549ac2b-8c60-40f7-802a-0edde18cc9d3,1,5e5ab8a09da33c5135fa5a39,National Tactics Administrator,1,2013-12-27 09:28:22,2017-10-23 16:03:16,0.0,Kirlin-Willms,2020-04-12 18:00:00,Dynamic Tactics Strategist,2020-04-12 08:00:00,0


In [15]:
df5_users=pd.merge(df3,df4,on='user_id',how='left')
#df5_user=df5_users[['address','mobile_no','dob','first_name','last_name','lat','long','skill_id','user_id','lang_id','Years of Experience','title']]
df6_users=pd.merge(df5_users,df_JobProposal,on='user_id',how='left')
df6_users.head()

,_id_x_x,__v_x_x,ip_address,updated_at_x_x,lat,created_at_x_x,status_x,email,password,dob,...,title,start_date,Experience in Months,_id,created_at,updated_at,__v,job_id,propoal_description,status_y
0,7d321f5c-36af-42ba-adb1-51f0c354505c,0.0,10.32.12.213,2018-07-22 03:00:12,46.216667,2014-05-16 18:09:18,1.0,kellen@zemlak.biz,98a04fa6-8c09-4e69-8aae-cf1303a3c601,1985-03-01,...,Customer Metrics Analyst,2019-04-12 08:00:00,12.0,5e5ab8a19da33c5135fa892f,2019-11-22 02:57:28,2012-08-14 13:32:37,0.0,e3b3470d-76f6-45c1-8afb-51035151e21c,Dynamic Interactions Facilitator,1
1,326791f7-69fc-4bf8-bfb2-4efaac0856c1,0.0,10.32.12.213,2011-11-13 06:50:47,46.216667,2015-08-13 19:43:19,1.0,anahi@jewess.biz,228d443c-8b92-4d02-b6b9-09340e1adfb9,1985-09-01,...,Global Markets Officer,2019-04-12 08:00:00,12.0,5e5ab8a19da33c5135fa8930,2010-08-12 12:41:15,2012-07-10 07:03:20,0.0,0e4db772-d915-4a4f-ad7e-65885a361744,Human Solutions Coordinator,1
2,65dac957-39bf-4d4a-b011-5e00f9d2417d,0.0,10.32.12.213,2011-11-22 23:13:59,46.216667,2019-02-13 05:41:22,1.0,simone@gusikowski.biz,7d1c84c5-8489-474d-9232-0dad56a7a438,1995-09-01,...,Dynamic Data Facilitator,2020-04-12 08:00:00,0.0,5e5ab8a19da33c5135fa8931,2017-06-13 19:54:06,2014-09-25 21:35:08,0.0,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,Dynamic Factors Facilitator,1
3,a74932df-7e7d-4572-a4f6-f738de6c2d31,0.0,10.32.12.213,2010-02-05 07:10:49,46.216667,2017-04-30 05:37:14,1.0,arvilla@douglaskassulke.name,fd590e52-954f-40ec-9ac6-c85dc01e2150,2001-09-01,...,Global Response Specialist,2020-04-12 08:00:00,0.0,5e5ab8a19da33c5135fa8932,2016-03-02 13:57:23,2012-06-06 13:18:57,0.0,96d0688a-204d-4529-9da5-34f653fec203,Investor Quality Strategist,1
4,c549ac2b-8c60-40f7-802a-0edde18cc9d3,0.0,10.32.12.213,2013-12-27 09:28:22,46.216667,2017-10-23 16:03:16,1.0,johann_schneider@grady.biz,7f66f005-9412-49ac-826b-1558db5606da,1985-09-01,...,Dynamic Tactics Strategist,2020-04-12 08:00:00,0.0,5e5ab8a19da33c5135fa8933,2018-06-01 08:46:34,2014-02-10 09:46:42,0.0,599334aa-c6d9-4306-aff9-5faccaf08c6f,Senior Creative Engineer,1


In [16]:
#Calculating age from date of birth column from df6_users
def calculate_age(born):
    born = datetime.strptime(born, "%Y-%m-%d").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
df6_users['age'] = df6_users['dob'].apply(calculate_age)

In [17]:
df6_users=df6_users[['lat','rate_x','long','_id_x_x','Experience in Months','skill_id','job_id','age','lang_id','description','propoal_description','title']]
df6_users.columns=['User_lat','Skill_rate','User_long','User_Id','Job Experience in months','Skill_Id','Job_Id','Age','Lang_Id','Description','Proposal desc','Title']
df6_users.head()


,User_lat,Skill_rate,User_long,User_Id,Job Experience in months,Skill_Id,Job_Id,Age,Lang_Id,Description,Proposal desc,Title
0,46.216667,1,-72.616669,7d321f5c-36af-42ba-adb1-51f0c354505c,12.0,e976c9af-784a-4874-8e3a-2b13c3017ecf,e3b3470d-76f6-45c1-8afb-51035151e21c,35,b60c3f90-5391-43a5-ab96-f6eb65dd80df,Investor Optimization Representative,Dynamic Interactions Facilitator,Customer Metrics Analyst
1,46.216667,4,-72.616669,326791f7-69fc-4bf8-bfb2-4efaac0856c1,12.0,84f9eb38-dd84-42c0-b3b9-719c019a7f00,0e4db772-d915-4a4f-ad7e-65885a361744,34,b769ad2b-a4e7-44f1-9a16-f9379e755ba4,Dynamic Security Director,Human Solutions Coordinator,Global Markets Officer
2,46.216667,3,-72.616669,65dac957-39bf-4d4a-b011-5e00f9d2417d,0.0,a2eae9ba-314e-418e-997a-909b7e365997,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,24,86582b97-988e-4c7b-8930-8e4a36be275a,Human Security Supervisor,Dynamic Factors Facilitator,Dynamic Data Facilitator
3,46.216667,2,-72.616669,a74932df-7e7d-4572-a4f6-f738de6c2d31,0.0,194ed753-2ebb-4451-95b8-a491a3cf49b4,96d0688a-204d-4529-9da5-34f653fec203,18,a1d428c2-ea35-4270-89eb-030fa4348aa6,Forward Response Facilitator,Investor Quality Strategist,Global Response Specialist
4,46.216667,2,-72.616669,c549ac2b-8c60-40f7-802a-0edde18cc9d3,0.0,87853669-bbd5-49d0-a542-358024a83e16,599334aa-c6d9-4306-aff9-5faccaf08c6f,34,e18a078d-9576-4cdb-a619-3790430b85d6,National Tactics Administrator,Senior Creative Engineer,Dynamic Tactics Strategist


In [18]:
df6_users["Job Experience in months"].fillna(0, inplace = True)
df6_users["Skill_Id"].fillna("Null", inplace = True)
df6_users["Description"].fillna("Null", inplace = True)
df6_users["Title"].fillna("Null", inplace = True)
#df6_users["User_Id"].fillna("Null", inplace = True)
df6_users["Lang_Id"].fillna("Null", inplace = True)
df6_users["Job_Id"].fillna("Null", inplace = True)
df6_users["Proposal desc"].fillna("Null", inplace = True)
len(df6_users)


10000

# Final dataframe

In [19]:
#i.	We have further mergerd df2_jobs and df6_users on job_id in order to use it as a dataframe to generate the final output  for a model.
df_final=pd.merge(df6_users,df2_jobs,on='Job_Id',how='left')
df_final.head().transpose()


,0,1,2,3,4
User_lat,46.216667,46.216667,46.216667,46.216667,46.216667
Skill_rate_x,1,4,3,2,2
User_long,-72.616669,-72.616669,-72.616669,-72.616669,-72.616669
User_Id,7d321f5c-36af-42ba-adb1-51f0c354505c,326791f7-69fc-4bf8-bfb2-4efaac0856c1,65dac957-39bf-4d4a-b011-5e00f9d2417d,a74932df-7e7d-4572-a4f6-f738de6c2d31,c549ac2b-8c60-40f7-802a-0edde18cc9d3
Job Experience in months,12,12,0,0,0
Skill_Id_x,e976c9af-784a-4874-8e3a-2b13c3017ecf,84f9eb38-dd84-42c0-b3b9-719c019a7f00,a2eae9ba-314e-418e-997a-909b7e365997,194ed753-2ebb-4451-95b8-a491a3cf49b4,87853669-bbd5-49d0-a542-358024a83e16
Job_Id,e3b3470d-76f6-45c1-8afb-51035151e21c,0e4db772-d915-4a4f-ad7e-65885a361744,36a5f6b6-18aa-4c4b-acae-2911cf05dfbf,96d0688a-204d-4529-9da5-34f653fec203,599334aa-c6d9-4306-aff9-5faccaf08c6f
Age,35,34,24,18,34
Lang_Id,b60c3f90-5391-43a5-ab96-f6eb65dd80df,b769ad2b-a4e7-44f1-9a16-f9379e755ba4,86582b97-988e-4c7b-8930-8e4a36be275a,a1d428c2-ea35-4270-89eb-030fa4348aa6,e18a078d-9576-4cdb-a619-3790430b85d6
Description,Investor Optimization Representative,Dynamic Security Director,Human Security Supervisor,Forward Response Facilitator,National Tactics Administrator


In [20]:
df_final["Skill_rate_y"].fillna(0, inplace = True)
df_final["Job_Status"].fillna("Null", inplace = True)
df_final["Total_salary"].fillna(0, inplace = True)
df_final["Job_Title"].fillna("Null", inplace = True)
#df6_users["User_Id"].fillna("Null", inplace = True)
df_final["Company_id"].fillna("Null", inplace = True)
df_final["Company_name"].fillna("Null", inplace = True)
#df_final["Company_lat"].fillna("Null", inplace = True)
df_final["Postal"].fillna("Null", inplace = True)
#df_final["Company_long"].fillna("Null", inplace = True)
df_final["Skill_Id_y"].fillna("Null", inplace = True)
df_final["Job_Description"].fillna("Null", inplace = True)
df_final.isna().sum()

User_lat                       0
Skill_rate_x                   0
User_long                      0
User_Id                        0
Job Experience in months       0
Skill_Id_x                     0
Job_Id                         0
Age                            0
Lang_Id                        0
Description                    0
Proposal desc                  0
Title                          0
Skill_rate_y                   0
Job_Status                     0
Total_salary                   0
Job_Title                      0
Company_id                     0
Company_name                   0
Company_lat                 9000
Postal                         0
Company_long                9000
Skill_Id_y                     0
Job_Description                0
dtype: int64

In [21]:
df_final["User_lat"]=pd.to_numeric(df_final["User_lat"])
df_final["User_long"]=pd.to_numeric(df_final["User_long"])
df_final["Company_lat"]=pd.to_numeric(df_final["Company_lat"])
df_final["Company_long"]=pd.to_numeric(df_final["Company_long"])

# Calculating Distance

In [22]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

for index, row in df_final.iterrows():
    df_final.loc[index, 'Distance in km'] = haversine(row['Company_long'], row['Company_lat'], row['User_long'], row['User_lat'])

In [23]:
#Replacing the Nan values if any from the column and then selecting final columns of the dataframe which we will be using in model
df_final['Distance in km'].fillna("Null", inplace = True)
df_final=df_final[['Skill_rate_x','User_Id','Job Experience in months','Skill_Id_x','Job_Id','Age','Lang_Id','Job_Description','Description','Title','Job_Title','Skill_rate_y','Proposal desc','Total_salary','Distance in km','Job_Status','Company_id','Company_name','Postal']]

# Splitting data into test and train dataset

In [24]:
#Splitting the users data in test and train
df_users_train, df_users_test=train_test_split(df6_users, test_size=0.2)

In [25]:
#length of the users train dataset
len(df_users_train)

8000

In [26]:
#length of the users test dataset
len(df_users_test)

2000

In [27]:
#Split the jobs dataset in 80 and 20
df_jobs_train, df_jobs_test = train_test_split(df2_jobs, test_size=0.2)

In [28]:
#length of the jobs train dataset
len(df_jobs_train)

8000

In [29]:
#length of the jobs test dataset
len(df_jobs_test)

2000

In [30]:
#taking input from title,description and proposal desc in df6_users['Description'] and filling up all the empty values

df_users_train['Job Experience in months']=df_users_train['Job Experience in months'].fillna("").astype('str')
df_users_train['Desc'] =df_users_train['Description']+" "+df_users_train['Title']+" "+df_users_train['Proposal desc']+df_users_train['Job Experience in months']
df_users_train['Desc'] = df_users_train['Desc'].fillna('')


In [31]:
#In order to judge machin's performance qualitatively, using TfidfVectorizer function from scikit-learn,
# which transforms text to feature vectors that can be used as input to estimator ,removing the stop words
# and computing TF-IDF matrix required for calculating cosine similarity and dispalying the shape of our matrix.
tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_users_train['Desc'])
tfidf_matrix.shape
#tf.get_feature_names()

(8000, 4656)

In [32]:
print(tfidf_matrix)

  (0, 3357)	0.35355339059327373
  (0, 3358)	0.35355339059327373
  (0, 3356)	0.35355339059327373
  (0, 3359)	0.35355339059327373
  (0, 3355)	0.7071067811865475
  (1, 3357)	0.35355339059327373
  (1, 3358)	0.35355339059327373
  (1, 3356)	0.35355339059327373
  (1, 3359)	0.35355339059327373
  (1, 3355)	0.7071067811865475
  (2, 3357)	0.35355339059327373
  (2, 3358)	0.35355339059327373
  (2, 3356)	0.35355339059327373
  (2, 3359)	0.35355339059327373
  (2, 3355)	0.7071067811865475
  (3, 3357)	0.35355339059327373
  (3, 3358)	0.35355339059327373
  (3, 3356)	0.35355339059327373
  (3, 3359)	0.35355339059327373
  (3, 3355)	0.7071067811865475
  (4, 3357)	0.35355339059327373
  (4, 3358)	0.35355339059327373
  (4, 3356)	0.35355339059327373
  (4, 3359)	0.35355339059327373
  (4, 3355)	0.7071067811865475
  :	:
  (7995, 3357)	0.35355339059327373
  (7995, 3358)	0.35355339059327373
  (7995, 3356)	0.35355339059327373
  (7995, 3359)	0.35355339059327373
  (7995, 3355)	0.7071067811865475
  (7996, 3357)	0.35355339

In [33]:
# computing cosine similarity matrix using linear_kernal of sklearn
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
cosine_sim[0]

array([1., 1., 1., ..., 1., 1., 1.])

In [34]:
#Index will be created here for the user_id
df= df_users_train.reset_index(drop=True)
id = df_users_train['User_Id']
indices = pd.Series(df.index,index=df_users_train['User_Id'])
indices.head()

User_Id
c1b213ef-c82b-4144-ad59-54ac4cc2eeaa    0
efafdd99-5bb8-4c27-83f4-8c9fd1c1d99f    1
5ea2b02a-d3c4-488f-86f0-46a9ac01fd92    2
54ef2167-e99a-4779-a658-40df798fb576    3
a9c44e92-4bb1-4edf-a7c0-d669ca13430f    4
dtype: int64

# Getting similar users

In [35]:
#function to get most similar users
def get_recommendations_userwise(userid):
    idx = indices[userid]
    #print (idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print (sim_scores)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores=sim_scores[1:numberOfRows]
    job_indices = [i[0] for i in sim_scores]
    return id.iloc[job_indices]

In [36]:
get_recommendations_userwise('5e5067b8-ce9e-437e-b643-8e9ac999a68f')

2623    efafdd99-5bb8-4c27-83f4-8c9fd1c1d99f
2364    5ea2b02a-d3c4-488f-86f0-46a9ac01fd92
8053    54ef2167-e99a-4779-a658-40df798fb576
8690    a9c44e92-4bb1-4edf-a7c0-d669ca13430f
7535    e398e11b-9d0b-4da0-b765-f22c4d50165f
Name: User_Id, dtype: object

In [37]:
print('User_Job_Title: ')
df_users_train['Title'].loc[df_users_train['User_Id'] == 'dae4b5df-752d-4f53-93c1-3a203c5a15fc']

User_Job_Title: 


5141    Null
Name: Title, dtype: object

# Recommending jobs to users

In [38]:
#function to recommend jobs to user by matching skill_id and filtering the output with distance
def get_job_id(usrid_list):
    jobs_userwise = df_users_train['User_Id'].isin(usrid_list)
    df1 = pd.DataFrame(data = df_users_train[jobs_userwise], columns=['Skill_Id'])
    joblist = df1['Skill_Id'].tolist()
    Job_list = df2_jobs['Skill_Id'].isin(joblist) #[1083186, 516837, 507614, 754917, 686406, 1058896, 335132])
    df_temp = pd.DataFrame(data = df_final[Job_list],columns=['User_Id','Job_Id','Distance in km','Company_name','Postal','Total_salary','Job_Description','Title'])
    return df_temp.loc[(df_temp['Distance in km'] <=distanceRange)]

In [39]:
get_job_id(get_recommendations_userwise('6d9daf03-6e84-4919-a7c5-53da5adbc4c2'))

,User_Id,Job_Id,Distance in km,Company_name,Postal,Total_salary,Job_Description,Title
17,83e83e7d-3b24-4cf6-9a25-220b174da4b6,5ef5a3a2-ab7a-4121-ad72-58a2f6d500de,0,"Torphy, Botsford and Emmerich",75338-2286,80,Automated responsive moratorium,Direct Quality Analyst
78,f3935b06-e169-4222-b303-2a1b7851d108,eb51e471-ceaa-44d5-b944-1cc0683f4716,0,Luettgen-Heaney,85547-8053,80,Vision-oriented well-modulated neural-net,Direct Quality Designer
85,db8d331e-8653-4945-ba5e-4a8d696fa8cf,da852d33-31d4-45a0-bf2b-fe00bf4c2daa,0,Tremblay LLC,30684-8141,80,Reduced 4th generation artificial intelligence,Central Accountability Administrator


In [40]:
df_users_train.tail(100)

,User_lat,Skill_rate,User_long,User_Id,Job Experience in months,Skill_Id,Job_Id,Age,Lang_Id,Description,Proposal desc,Title,Desc
5207,46.216667,4,-72.616669,2ad8921f-28dc-47a4-9f01-6938c4f25284,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
4379,46.216667,3,-72.616669,914e5ffd-cf60-44ea-b860-caaa607e0eb2,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
6206,46.216667,4,-72.616669,978eab66-6960-4cc9-91b8-6c39ad97b536,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
5958,46.216667,3,-72.616669,e552bd2b-11a3-4423-b0fa-49dac5845ede,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
548,46.216667,1,-72.616669,60354b93-01e6-4316-b324-add5d170f92b,0.0,80d40f9b-22c3-4671-888e-c0d5ed6387c3,74d0d5ad-e163-4681-9841-b331b1c5118f,34,3b4ffe38-6dd1-402c-9d5c-6e2a0bc87925,Internal Creative Developer,Internal Security Director,Lead Solutions Agent,Internal Creative Developer Lead Solutions Age...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9777,46.216667,5,-72.616669,16055545-6729-43ee-ba97-22199c05aa46,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
8320,46.216667,4,-72.616669,df4c36d6-6a59-439f-b0e7-c01b86375c76,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
3020,46.216667,1,-72.616669,45b9ade2-179f-4f60-81fc-53198a83e14e,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0
7645,46.216667,3,-72.616669,f6bdabe7-1578-4e95-a9d9-2c34c11b50b3,0.0,Null,Null,34,Null,Null,Null,Null,Null Null Null0.0


# Recommending users to companies

In [41]:
#taking input from title,description and Skill_rate in df2_jobs['Description'] and filling up all the empty values
df2_jobs['Skill_rate']=df2_jobs['Skill_rate'].fillna("").astype('str')
df2_jobs['Descr'] = df2_jobs['Job_Description']+" "+df2_jobs['Job_Title']+" "+df2_jobs['Skill_rate']
df2_jobs['Descr'] = df2_jobs['Job_Description'] .fillna('')

In [42]:
#In order to judge machin's performance qualitatively, using TfidfVectorizer function from scikit-learn,
# which transforms text to feature vectors that can be used as input to estimator ,removing the stop words
# and computing TF-IDF matrix required for calculating cosine similarity and dispalying the shape of our matrix.
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(df2_jobs['Descr'])
tfidf_matrix1.shape

(10000, 1187)

In [43]:
# computing cosine similarity matrix using linear_kernal of sklearn
cosine_simi = linear_kernel(tfidf_matrix1,tfidf_matrix1)
cosine_simi[0]

array([1.        , 0.        , 0.        , ..., 0.10999557, 0.        ,
       0.        ])

In [44]:
#Index will be created here for the Job_Id
df2_jobs = df2_jobs.reset_index(drop=True)
jobid = df2_jobs['Job_Id']
indices1 = pd.Series(df2_jobs.index,index=df2_jobs['Job_Id'])
indices1.head(2)

Job_Id
e3b3470d-76f6-45c1-8afb-51035151e21c    0
0e4db772-d915-4a4f-ad7e-65885a361744    1
dtype: int64

In [45]:
#function fro similar job_ids
def get_recommendations(job_id):
    jobidx = indices1[job_id]
    #print (idx)
    sim_scores1 = list(enumerate(cosine_simi[jobidx]))
    #print (sim_scores1)
    sim_scores1= sorted(sim_scores1, key=lambda x: x[1], reverse=True)
    sim_scores1=sim_scores1[1:numberOfRows]
    user_indices = [i[0] for i in sim_scores1]
    return jobid.iloc[user_indices]

In [46]:
get_recommendations("9ba580f8-1e27-4a92-a457-2d4f4409bbda")

590     4476ee33-f67c-4505-97f9-0da2111fa1f8
990     0b3573f3-c209-4f25-b301-d441392261a1
1390    9cc5255e-4d0f-4d44-b164-b31f462abd9f
1790    af3493b4-ecd1-4387-b673-962d185356c4
2190    278f74d8-2772-4882-9b64-5c628ef9956c
Name: Job_Id, dtype: object

In [47]:
print('User_Job_Description: ')
df2_jobs['Job_Description'].loc[df2_jobs['Job_Id'] == 'c76be6a4-0761-4216-a5fa-52446e28e62b']

User_Job_Description: 


560    Multi-lateral needs-based conglomeration
Name: Job_Description, dtype: object

In [48]:
#function to recommend users for the job_id 
def get_user_id(jobid_list):
    user_jobwise = df_users_train['Job_Id'].isin(jobid_list)
    df_1 = pd.DataFrame(data = df_users_train[user_jobwise], columns=['Skill_Id'])
    userlist = df_1['Skill_Id'].tolist()
    User_list = df2_jobs['Skill_Id'].isin(userlist) #[1083186, 516837, 507614, 754917, 686406, 1058896, 335132])
    df_tmp = pd.DataFrame(data = df_final[User_list],columns=['Job_Id','User_Id','Skill_Id_x','Job_Title','Age','Job_Description','Job Experience in months'])
    return df_tmp#.loc[(df_tmp['Skill_rate_x']>=killRateRange)]


In [49]:
get_user_id(get_recommendations("c76be6a4-0761-4216-a5fa-52446e28e62b"))

,Job_Id,User_Id,Skill_Id_x,Job_Title,Age,Job_Description,Job Experience in months
60,0fe95be4-775b-4d72-a82b-be80c72021f0,c95cb11d-5d3d-4f0e-9ad3-6c353e289e6d,158c86ed-d1fa-4a96-b6c4-822545dfddf3,3:Casinos:,34,Distributed intangible hierarchy,0.0
